In [ ]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from sqlalchemy import create_engine, text
import re

In [ ]:
username = 'admin'
password = 'nX0d4tuvxL3w5hMjJRfz'
host = 'ai-engine.cluster-cla9jbugaywl.eu-west-2.rds.amazonaws.com'
database_name = 'superleague_clubs'

connection_url = f'mysql+pymysql://{username}:{password}@{host}/{database_name}'

In [ ]:
def insert_into_db(df, table_name) : 

    engine = create_engine(connection_url)
    df.to_sql(table_name, con=engine, if_exists='replace', index=False) 
    return 'Data inserted successfully'

In [ ]:
engine = create_engine(connection_url)

club = 'villarreal'
query = f'select distinct playername, player_url from `transfermarkt_players_fc-villarreal`;'
sql_text = text(query)
with engine.connect() as connection:
    result = connection.execute(sql_text)
    result_list = result.fetchall()

engine.dispose()    

In [ ]:
def extractID(playerURL):

    match = re.search(r'/(\d+)$', playerURL)

    if match:
        extracted_id = match.group(1)
        return extracted_id

In [ ]:
data = []
headers = ['player', 'season', 'date', 'from', 'to', 'mv', 'fee', 'tm_playerid']

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu") 
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"
chrome_options.add_argument(f"user-agent={user_agent}")

for player, url in result_list : 

    player_url = 'https://www.transfermarkt.com' + url
    player_id = extractID(player_url)
    print(player_url)

    try : 
        driver = webdriver.Chrome(options=chrome_options)  
        driver.get(player_url)
        driver.implicitly_wait(10)

        page_source = driver.page_source
        driver.quit()

        soup = BeautifulSoup(page_source, "lxml")

        th = soup.find_all('div', class_='grid tm-player-transfer-history-grid')

        for row in th:
            
            rowdata = []
            rowdata.append(player)
            rowdata.append(row.find('div', class_='grid__cell grid__cell--center tm-player-transfer-history-grid__season').text.strip())
            rowdata.append(row.find('div', class_='grid__cell grid__cell--center tm-player-transfer-history-grid__date').text.strip())
            rowdata.append(row.find('div', class_='grid__cell grid__cell--center tm-player-transfer-history-grid__old-club').text.strip())
            rowdata.append(row.find('div', class_='grid__cell grid__cell--center tm-player-transfer-history-grid__new-club').text.strip())
            rowdata.append(row.find('div', class_='grid__cell grid__cell--center tm-player-transfer-history-grid__market-value').text.strip())
            rowdata.append(row.find('div', class_='grid__cell grid__cell--center tm-player-transfer-history-grid__fee').text.strip())
            rowdata.append(player_id)
            data.append(rowdata)

    except Exception as e : 
        print(f'Could not get data for player {player_url}')

final_df = pd.DataFrame(data, columns=headers)
insert_into_db(final_df, f'{club}_transferhistory')

In [6]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
from selenium import webdriver


url = 'https://www.transfermarkt.com/jose-antonio-navarro/profil/spieler/462527'

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu") 

driver = webdriver.Chrome()  
driver.get(url)
driver.implicitly_wait(3)

page_source = driver.page_source
driver.quit()

page_source = driver.page_source

soup = BeautifulSoup(page_source, "lxml")
th = soup.find_all('div', class_='grid tm-player-transfer-history-grid')

MaxRetryError: HTTPConnectionPool(host='localhost', port=62056): Max retries exceeded with url: /session/d5b2fba7b499ae318315a3f7288460cc/source (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x10d0caf80>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [ ]:
th